In [1]:
import os, glob, warnings, re, shutil
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from scipy.stats import zscore

import numpy as np
from scipy import interpolate


In [2]:
# prepare spots
spot_list = ['commercial_aribnb_30s', 'commercial_att_30s', 'commercial_carscom_30s','commercial_cookies_30s',
             'commercial_dominos_30s', 'commercial_doritos_30s','commercial_expedia_30s','commercial_google_pixel_30s',
             'commercial_hr_block_30s','commercial_jersey_mikes_30s','commercial_lego_30s','commercial_meta_quest_2_30s',
             'commercial_milk_30s','commercial_progressive_30s','commercial_publix_30s','commercial_puma_30s',
             'commercial_starbucks_30s','commercial_under_armour_30s',
             'health_alcohol_30s','health_alzheimers_30s','health_covid_vaccine_30s','health_diet_30s',
             'health_drunk_driving_30s','health_fitness_30s','health_kidney_30s','health_mantherapy_30s',
             'health_prediabetes_30s','health_stroke_30s','health_vaping_30s','health_weight_30s']

spot_list.sort()
print(len(spot_list))
print(spot_list[:3])

# prepare subject list
sub_folders = glob.glob("../data/00_raw_data/sub*")
sub_folders.sort()
subject_list = []
for f in sub_folders:
    subject_list.append(f[-6:])

print(len(subject_list))
print(subject_list[:3])

# prepare (read in) memory data
memory_df = pd.read_csv('../data/00_memory_data/recall_vr_video.csv')
memory_df.head()

30
['commercial_aribnb_30s', 'commercial_att_30s', 'commercial_carscom_30s']
59
['sub001', 'sub002', 'sub003']


,subject,commercial_aribnb_30s,commercial_carscom_30s,commercial_cookies_30s,commercial_dominos_30s,commercial_doritos_30s,commercial_expedia_30s,commercial_google_pixel_30s,commercial_hr_block_30s,commercial_jersey_mikes_30s,...,health_diet_30s,health_drunk_driving_30s,health_fitness_30s,health_kidney_30s,health_mantherapy_30s,health_prediabetes_30s,health_alcohol_30s,health_weight_30s,health_stroke_30s,health_vaping_30s
0,sub001,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,1,0,0
1,sub002,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
2,sub003,1,1,0,0,1,0,0,0,1,...,1,1,0,1,1,1,1,0,0,1
3,sub004,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,1,0,0
4,sub005,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
memory_df.columns

Index(['subject', 'commercial_aribnb_30s', 'commercial_carscom_30s',
       'commercial_cookies_30s', 'commercial_dominos_30s',
       'commercial_doritos_30s', 'commercial_expedia_30s',
       'commercial_google_pixel_30s', 'commercial_hr_block_30s',
       'commercial_jersey_mikes_30s', 'commercial_lego_30s',
       'commercial_meta_quest_2_30s', 'commercial_milk_30s',
       'commercial_att_30s', 'commercial_progressive_30s',
       'commercial_publix_30s', 'commercial_puma_30s',
       'commercial_starbucks_30s', 'commercial_under_armour_30s',
       'health_alzheimers_30s', 'health_covid_vaccine_30s', 'health_diet_30s',
       'health_drunk_driving_30s', 'health_fitness_30s', 'health_kidney_30s',
       'health_mantherapy_30s', 'health_prediabetes_30s', 'health_alcohol_30s',
       'health_weight_30s', 'health_stroke_30s', 'health_vaping_30s'],
      dtype='object')

#### loop over spots and subjects, copying the pupil data and annotating them by recall status for future use

In [4]:
for current_spot   in spot_list:
    print(current_spot)
    for current_subject in subject_list:
        #print(current_subject)
        #current_subject = 'sub001'

        if (memory_df[ memory_df['subject'] == current_subject][current_spot].values[0] == 0):
            recall_status = 'recall0'
        elif (memory_df[ memory_df['subject'] == current_subject][current_spot].values[0] == 1):
            recall_status = 'recall1'

        #print(recall_status)

        ### pupil file for this participant and spot (source file)
        file_to_find = '../data/01_parsed_video_data/' + current_subject + '/' + current_spot +'*'
        #print(file_to_find)

        file_to_copy = glob.glob(file_to_find)
        #print(file_to_copy[0])
        if (len(file_to_copy) >0):

            if re.search('_100.csv', file_to_copy[0]):
                    condition = '100'
            elif re.search('_50nd.csv', file_to_copy[0]):
                    condition = '50nd'
            elif re.search('_50wd.csv', file_to_copy[0]):
                    condition = '50wd'

            ### new file path for this partipant and file
            destination_folder = '../data/03_spots_memory_data/' + current_spot + '/' + condition + '/' 
            isExist = os.path.exists(destination_folder)
            if not isExist:
               # Create a new directory because it does not exist
               os.makedirs(destination_folder)
               #print("The new directory is created!")

            destination_path = destination_folder + current_subject + '_' + recall_status + '.csv'
            #print(destination_path)

            shutil.copyfile(file_to_copy[0], destination_path)


commercial_aribnb_30s
commercial_att_30s
commercial_carscom_30s
commercial_cookies_30s
commercial_dominos_30s
commercial_doritos_30s
commercial_expedia_30s
commercial_google_pixel_30s
commercial_hr_block_30s
commercial_jersey_mikes_30s
commercial_lego_30s
commercial_meta_quest_2_30s
commercial_milk_30s
commercial_progressive_30s
commercial_publix_30s
commercial_puma_30s
commercial_starbucks_30s
commercial_under_armour_30s
health_alcohol_30s
health_alzheimers_30s
health_covid_vaccine_30s
health_diet_30s
health_drunk_driving_30s
health_fitness_30s
health_kidney_30s
health_mantherapy_30s
health_prediabetes_30s
health_stroke_30s
health_vaping_30s
health_weight_30s
